# How to use this notebook

1. install requirements with ```pip install -r requirements.txt```
2. start mlflow server ```mlflow server --port=5000```
3. run through this notebook

In [1]:
import pandas as pd

# Dataset Prep

The goal is to merge calendar, price and sales data into one frame which will be used by the model afterwards. Static predictors that are only dependent on the id are extracted separately

## Calendar

For the calendar data, the weekdays are ordinally encoded while all of the events and types are one hot encoded

In [4]:
calendar = pd.read_csv("./data/unprocessed/calendar.csv")
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [5]:
calendar['weekday'] = calendar.weekday.apply(lambda x: ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'].index(x))

In [6]:
event1 = pd.get_dummies(calendar.event_name_1)
event2 = pd.get_dummies(calendar.event_name_2)
# merge overlapping columns by summing up
event1["Cinco De Mayo"] = event2["Cinco De Mayo"] + event1["Cinco De Mayo"]
event1["Easter"] = event2["Easter"] + event1["Easter"]
event1["Father's day"] = event2["Father's day"] + event1["Father's day"]
event1["OrthodoxEaster"] = event2["OrthodoxEaster"] + event1["OrthodoxEaster"]

type1 = pd.get_dummies(calendar.event_type_1)
type2 = pd.get_dummies(calendar.event_type_2)
# merge overlapping columns by summing up
type1['Cultural'] = type2.Cultural + type1.Cultural
type1['Religious'] = type2.Religious + type1.Religious

In [7]:
calendar_fixed = pd.concat([calendar, event1, type1], axis=1).drop(['event_name_1', 'event_name_2', 'event_type_1', 'event_type_2'], axis=1)
calendar_fixed

,date,wm_yr_wk,weekday,wday,month,year,d,snap_CA,snap_TX,snap_WI,...,Ramadan starts,StPatricksDay,SuperBowl,Thanksgiving,ValentinesDay,VeteransDay,Cultural,National,Religious,Sporting
0,2011-01-29,11101,5,1,1,2011,d_1,0,0,0,...,False,False,False,False,False,False,False,False,False,False
1,2011-01-30,11101,6,2,1,2011,d_2,0,0,0,...,False,False,False,False,False,False,False,False,False,False
2,2011-01-31,11101,0,3,1,2011,d_3,0,0,0,...,False,False,False,False,False,False,False,False,False,False
3,2011-02-01,11101,1,4,2,2011,d_4,1,1,0,...,False,False,False,False,False,False,False,False,False,False
4,2011-02-02,11101,2,5,2,2011,d_5,1,0,1,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,2,5,6,2016,d_1965,0,1,1,...,False,False,False,False,False,False,False,False,False,False
1965,2016-06-16,11620,3,6,6,2016,d_1966,0,0,0,...,False,False,False,False,False,False,False,False,False,False
1966,2016-06-17,11620,4,7,6,2016,d_1967,0,0,0,...,False,False,False,False,False,False,False,False,False,False
1967,2016-06-18,11621,5,1,6,2016,d_1968,0,0,0,...,False,False,False,False,False,False,False,False,False,False


## Prices

Prices are kept as they are. One column is added indicating if there was a price change in the respective week

In [8]:
prices = pd.read_csv("./data/unprocessed/sell_prices.csv")
prices['price_change'] = prices.groupby(['item_id']).sell_price.diff().apply(lambda x: 0 if x == 0 or pd.isna(x) else 1)
prices

,store_id,item_id,wm_yr_wk,sell_price,price_change
0,CA_1,HOBBIES_1_001,11325,9.58,0
1,CA_1,HOBBIES_1_001,11326,9.58,0
2,CA_1,HOBBIES_1_001,11327,8.26,1
3,CA_1,HOBBIES_1_001,11328,8.26,0
4,CA_1,HOBBIES_1_001,11329,8.26,0
...,...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00,0
6841117,WI_3,FOODS_3_827,11618,1.00,0
6841118,WI_3,FOODS_3_827,11619,1.00,0
6841119,WI_3,FOODS_3_827,11620,1.00,0


## Sales

Static predictors are extracted and sales data is merged with the other two frames

In [10]:
sales = pd.read_csv("./data/unprocessed/sales_train_evaluation.csv")
static_vars = sales[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']]

In [11]:
sales = sales.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='day', value_name='sales')
sales

,id,item_id,dept_id,cat_id,store_id,state_id,day,sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1
59181086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0


In [12]:
sales_cal = sales.merge(calendar_fixed, how='left', left_on='day', right_on='d').drop(['d'], axis=1)
sales_cal

,id,item_id,dept_id,cat_id,store_id,state_id,day,sales,date,wm_yr_wk,...,Ramadan starts,StPatricksDay,SuperBowl,Thanksgiving,ValentinesDay,VeteransDay,Cultural,National,Religious,Sporting
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,False,False,False,False,False,False,False,False,False,False
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,False,False,False,False,False,False,False,False,False,False
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,False,False,False,False,False,False,False,False,False,False
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,False,False,False,False,False,False,False,False,False,False
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1,2016-05-22,11617,...,False,False,False,False,False,False,False,False,False,False
59181086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,...,False,False,False,False,False,False,False,False,False,False
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2,2016-05-22,11617,...,False,False,False,False,False,False,False,False,False,False
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,...,False,False,False,False,False,False,False,False,False,False


Static predictors are removed here as well as date and the week indicator since they don't offer any additional information

In [13]:
sales_cal_price = sales_cal.merge(prices, how='left', on=['store_id', 'item_id', 'wm_yr_wk']).drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'date', 'wm_yr_wk'], axis=1)
sales_cal_price

,id,day,sales,weekday,wday,month,year,snap_CA,snap_TX,snap_WI,...,SuperBowl,Thanksgiving,ValentinesDay,VeteransDay,Cultural,National,Religious,Sporting,sell_price,price_change
0,HOBBIES_1_001_CA_1_evaluation,d_1,0,5,1,1,2011,0,0,0,...,False,False,False,False,False,False,False,False,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,d_1,0,5,1,1,2011,0,0,0,...,False,False,False,False,False,False,False,False,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,d_1,0,5,1,1,2011,0,0,0,...,False,False,False,False,False,False,False,False,NaN,NaN
3,HOBBIES_1_004_CA_1_evaluation,d_1,0,5,1,1,2011,0,0,0,...,False,False,False,False,False,False,False,False,NaN,NaN
4,HOBBIES_1_005_CA_1_evaluation,d_1,0,5,1,1,2011,0,0,0,...,False,False,False,False,False,False,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,d_1941,1,6,2,5,2016,0,0,0,...,False,False,False,False,False,False,False,False,2.98,0.0
59181086,FOODS_3_824_WI_3_evaluation,d_1941,0,6,2,5,2016,0,0,0,...,False,False,False,False,False,False,False,False,2.48,0.0
59181087,FOODS_3_825_WI_3_evaluation,d_1941,2,6,2,5,2016,0,0,0,...,False,False,False,False,False,False,False,False,3.98,0.0
59181088,FOODS_3_826_WI_3_evaluation,d_1941,0,6,2,5,2016,0,0,0,...,False,False,False,False,False,False,False,False,1.28,0.0


## Saving the data

Only worked with 1M data points for now since it quickly overwhelmed my pc

In [14]:
sales_cal_price[:1000000].to_csv('./data/processed/sales.csv')
static_vars.to_csv('./data/processed/static_vars.csv')

# Sampling test days

With this I can sample days that did have/didn't have price changes. Still need to figure out where to integrate this in the model code

In [ ]:
test_price_change = sales_cal_price[sales_cal_price.price_change == 1].sample(10000, random_state=42)
test_price_change

In [ ]:
test_no_price_change = sales_cal_price[sales_cal_price.price_change == 0].sample(10000, random_state=42)
test_no_price_change

# Run the model

I tried running it without sweeping but I get really weird errors which I seemingly can't fix so leaving it as is for now

In [ ]:
!set PYTHONPATH=. && set CUDA_VISIBLE_DEVICES=0 && python runnables/train_multi.py -m +dataset=m5 +backbone=ct +backbone/ct_hparams/cancer_sim_domain_conf='0' exp.seed=10